In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

In [2]:
df = pd.read_csv('../Datasets/reduced_II.csv')

In [3]:
df.head()

,P_Id,PPG_Rate_Mean,HRV_SDNN,HRV_MadNN,HRV_SDRMSSD,HRV_Prc20NN,HRV_MinNN,HRV_TINN,HRV_LF,HRV_HF,...,HRV_MFDFA_alpha2_Fluctuation,HRV_ApEn,HRV_SampEn,HRV_ShanEn,HRV_MSEn,HRV_CMSEn,HRV_HFD,HRV_KFD,HRV_LZC,anxiety_meter
0,101,72.160656,161.144697,106.0059,0.744888,754.2,308.0,593.7500,0.016549,0.110477,...,0.000557,0.905663,1.662548,7.103751,1.229793,1.392336,1.965384,4.166548,1.197715,7.0
1,102,94.333514,196.751139,152.7078,0.729774,519.4,307.0,320.3125,0.054757,0.067803,...,0.000133,1.094277,1.868132,7.555224,1.174007,1.288954,1.967041,2.788036,1.117846,16.0
2,103,91.269287,240.470162,203.1162,0.766559,473.0,307.0,203.1250,0.050173,0.072260,...,0.000077,0.962326,1.746639,7.316140,1.387296,1.315389,1.939231,4.690344,1.122028,10.0
3,104,76.862836,295.652405,182.3598,0.704668,600.6,309.0,414.0625,0.054656,0.072211,...,0.000351,0.891039,1.377711,7.143425,1.196983,1.302091,1.974733,2.787734,1.051012,8.0
4,105,68.409940,655.450522,324.6894,0.712167,564.0,301.0,453.1250,0.040793,0.017122,...,0.000442,1.097319,1.668343,7.084940,1.458232,1.302750,1.984928,2.193969,1.162819,15.0


In [4]:
df.shape

(101, 42)

In [5]:
X = df.drop(columns=['anxiety_meter', 'P_Id'], axis=1)
y = df['anxiety_meter']

In [6]:
X

,PPG_Rate_Mean,HRV_SDNN,HRV_MadNN,HRV_SDRMSSD,HRV_Prc20NN,HRV_MinNN,HRV_TINN,HRV_LF,HRV_HF,HRV_VHF,...,HRV_MFDFA_alpha2_Asymmetry,HRV_MFDFA_alpha2_Fluctuation,HRV_ApEn,HRV_SampEn,HRV_ShanEn,HRV_MSEn,HRV_CMSEn,HRV_HFD,HRV_KFD,HRV_LZC
0,72.160656,161.144697,106.0059,0.744888,754.2,308.0,593.7500,0.016549,0.110477,0.026186,...,-0.339827,0.000557,0.905663,1.662548,7.103751,1.229793,1.392336,1.965384,4.166548,1.197715
1,94.333514,196.751139,152.7078,0.729774,519.4,307.0,320.3125,0.054757,0.067803,0.018893,...,-0.585898,0.000133,1.094277,1.868132,7.555224,1.174007,1.288954,1.967041,2.788036,1.117846
2,91.269287,240.470162,203.1162,0.766559,473.0,307.0,203.1250,0.050173,0.072260,0.013437,...,-0.851135,0.000077,0.962326,1.746639,7.316140,1.387296,1.315389,1.939231,4.690344,1.122028
3,76.862836,295.652405,182.3598,0.704668,600.6,309.0,414.0625,0.054656,0.072211,0.010139,...,-0.542126,0.000351,0.891039,1.377711,7.143425,1.196983,1.302091,1.974733,2.787734,1.051012
4,68.409940,655.450522,324.6894,0.712167,564.0,301.0,453.1250,0.040793,0.017122,0.000683,...,0.000000,0.000442,1.097319,1.668343,7.084940,1.458232,1.302750,1.984928,2.193969,1.162819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,106.643368,199.166962,64.4931,0.811141,461.6,332.0,218.7500,0.048237,0.067233,0.010414,...,-0.712466,0.007395,0.789207,0.678190,6.976664,1.056822,1.177499,1.907768,2.744730,0.736001
97,88.979798,429.343966,123.7971,0.832398,480.8,350.0,195.3125,0.068547,0.014395,0.000205,...,-0.368860,0.017545,0.823860,0.770202,6.889927,0.837755,1.038622,1.897684,1.796485,1.018469
98,109.566313,155.993383,50.4084,0.795211,469.0,309.0,250.0000,0.046989,0.094851,0.029186,...,-0.708743,0.000805,0.856094,0.805586,7.184780,0.978753,1.191947,1.932554,2.855720,0.763747
99,126.073719,112.120465,34.8411,0.841414,427.0,302.0,171.8750,0.040691,0.030826,0.003372,...,-0.893389,0.001636,0.783548,0.744972,6.726446,0.474042,1.009792,1.917123,1.941572,0.839453


In [7]:
y = (y >= 16).astype(int)
y.name = 'IsAnxious'

In [8]:
y

0      0
1      1
2      0
3      0
4      0
      ..
96     1
97     1
98     1
99     1
100    0
Name: IsAnxious, Length: 101, dtype: int64

In [9]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [10]:
def create_pipeline(classifier):
    return Pipeline([
        ('imputer', KNNImputer(n_neighbors=5)),  # Replace missing values with the mean
        ('scaler', RobustScaler()),  # Scale features using RobustScaler
        ('kbest', SelectKBest(score_func=f_classif, k=15)),  # Select features using SelectKBest
        ('classifier', classifier),  # Classifier
    ])

# Define the parameter grid for GridSearchCV
param_grids = {
    'svc': {
        'classifier__kernel': ['linear', 'rbf'],
        'classifier__C': [0.1, 1.0, 10.0],
        'classifier__gamma': ['scale', 'auto']
    },
    'rfc': {
        'classifier': [RandomForestClassifier(random_state=42)],
        'classifier__n_estimators': [50, 100, 200],
        'classifier__max_depth': [None, 10, 20, 30],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__min_samples_leaf': [1, 2, 4],
        'classifier__max_features': ['sqrt', 'log2']
    },
    'logreg': {
        'classifier': [LogisticRegression(random_state=42)],
        'classifier__solver': ['lbfgs', 'liblinear'],
        'classifier__C': [0.1, 1.0, 10.0]
    },
    'xgboost': {
        'classifier': [XGBClassifier(random_state=42)],
        'classifier__learning_rate': [0.1, 0.01],
        'classifier__n_estimators': [100, 200, 300],
        'classifier__max_depth': [3, 5, 7]
    },
    'dtc': {
        'classifier': [DecisionTreeClassifier(random_state=42)],
        'classifier__max_depth': [None, 10, 20, 30],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__min_samples_leaf': [1, 2, 4],
        'classifier__max_features': ['sqrt', 'log2']
    },
    'nbayes': {
        'classifier': [GaussianNB()]
    },
    'voting-algorithm': {
        'classifier': [VotingClassifier(estimators=[('svc', SVC(random_state=42))])]
    }
}

# Create StratifiedKFold for cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform GridSearchCV for each classifier
results = {}
for model_name, param_grid in param_grids.items():
    if 'classifier' in param_grid:
        classifier = param_grid['classifier'][0]
        del param_grid['classifier']
        pipeline = create_pipeline(classifier)
    else:
        pipeline = create_pipeline(SVC(random_state=42))
    
    grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='accuracy', verbose=1, n_jobs=-1)
    grid_search.fit(X, y)  # Assuming X and y are your features and target variables
    results[model_name] = {
        'best_params': grid_search.best_params_,
        'best_score': grid_search.best_score_,
        'best_std_deviation': grid_search.cv_results_['std_test_score'][grid_search.best_index_]
    }

# Print results for each model
for model_name, result in results.items():
    print(f"Model: {model_name}")
    print("Best parameters:", result['best_params'])
    print("Best cross-validation score:", result['best_score'])
    print("Best standard deviation:", result['best_std_deviation'])
    print("-" * 50)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Model: svc
Best parameters: {'classifier__C': 0.1, 'classifier__gamma': 'scale', 'classifier__kernel': 'rbf'}
Best cross-validation score: 0.5742857142857142
Best standard deviation: 0.022406267344797285
--------------------------------------------------
Model: rfc
Best parameters: {'classifier__max_depth': None, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 4, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 50}
Best cross-validation score: 0.5147619047619048
Best standard deviation: 0.04894348631439845
--------------------------------------------------
Model: logreg
Best parameters: {'clas